In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from sklearn.linear_model import LinearRegression

path_CR_histo_regions = r"M:\PRICING\Pricer - Agrégation\benedrien\Dev Python\Parameters.xlsx"
sheet_source = "Histo CR régions"


Create one sheet for each region / techno + hpsn

In [ ]:

df = pd.read_excel(path_CR_histo_regions, sheet_name=sheet_source)

required_cols = ["libelle_region", "Mois-annee", "CR eol sans hpsn", "CR sol sans hpsn"]
for col in required_cols:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' missing from sheet '{sheet_source}'.")

# --- Extract Year / Month ---
df[["Year", "Month"]] = df["Mois-annee"].astype(str).str.split("-", expand=True)
df["Year"] = df["Year"].astype(int)
df["Month"] = df["Month"].astype(int)

# --- Month names ---
month_names = {1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
               7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"}
df["Month_name"] = df["Month"].map(month_names)

# --- Load workbook ---
wb = load_workbook(path_CR_histo_regions)
fill_forecast = PatternFill(start_color="FFFACD", end_color="FFFACD", fill_type="solid")  # yellow

# --- Function to write pivot sheet ---
def create_pivot_sheet(pivot_df, sheet_name, last_year=None, slopes=None, highlight_future=False, as_percent=False):
    if sheet_name in wb.sheetnames:
        del wb[sheet_name]
    ws = wb.create_sheet(title=sheet_name)
    ws.append(["Mois"] + list(pivot_df.columns) + ["", "Slope"])
    
    for i, (month, row) in enumerate(pivot_df.iterrows(), start=2):
        ws.cell(row=i, column=1, value=month)
        for j, (year, val) in enumerate(row.items(), start=2):
            if pd.isna(val):
                continue
            val_display = val * 100 if as_percent else val
            cell = ws.cell(row=i, column=j, value=val_display)
            if highlight_future and last_year is not None and year >= last_year:
                cell.fill = fill_forecast
        if slopes is not None:
            slope_val = slopes.get(month)
            if slope_val is not None and as_percent:
                slope_val *= 100
            ws.cell(row=i, column=len(pivot_df.columns)+3, value=slope_val)

# --- Process each region / technology ---
for region in df["libelle_region"].dropna().unique():
    sub = df[df["libelle_region"] == region].copy()
    
    for tech, col in [("Eolien", "CR eol sans hpsn"), ("Solaire", "CR sol sans hpsn")]:
        pivot = pd.pivot_table(sub, index="Month_name", columns="Year", values=col, aggfunc="mean")
        pivot = pivot.reindex(list(month_names.values()))  # ensure Jan-Dec order

        if pivot.empty:
            continue

        years = sorted(pivot.columns.tolist())
        last_year = max(years)
        future_years = list(range(last_year + 1, last_year + 6))
        for y in future_years:
            pivot[y] = np.nan

        slopes = {}
        for month in pivot.index:
            vals = pivot.loc[month, years].dropna()
            if len(vals) >= 2:
                X = np.array(vals.index).reshape(-1, 1)
                y_vals = vals.values
                model = LinearRegression().fit(X, y_vals)
                slope = model.coef_[0]
                slopes[month] = slope

                # Predict missing in last year + future
                pred_years = [y for y in years + future_years if pd.isna(pivot.loc[month, y])]
                if pred_years:
                    preds = model.predict(np.array(pred_years).reshape(-1, 1))
                    # Cap ±5% relative to last known
                    last_known_year = vals.index.max()
                    last_known_val = vals.loc[last_known_year]
                    preds_capped = []
                    for yhat in preds:
                        delta = (yhat - last_known_val) / last_known_val
                        if delta > 0.05:
                            yhat = last_known_val * 1.05
                        elif delta < -0.05:
                            yhat = last_known_val * 0.95
                        preds_capped.append(yhat)
                    for y_pred, y_val in zip(pred_years, preds_capped):
                        pivot.loc[month, y_pred] = y_val
            else:
                slopes[month] = None

        create_pivot_sheet(pivot, f"{tech}_{region}".replace("/", "_")[:31],
                           last_year=last_year, slopes=slopes,
                           highlight_future=True, as_percent=True)

region_hpsn = "Bretagne"
sub_hpsn = df[df["libelle_region"] == region_hpsn].copy()

if not sub_hpsn.empty and "nb hpsn" in sub_hpsn.columns:
    pivot_hpsn = pd.pivot_table(
        sub_hpsn,
        index="Month_name",
        columns="Year",
        values="nb hpsn",
        aggfunc="sum"  # or "mean" if you prefer
    ).reindex(list(month_names.values()))  # Jan → Dec

    # Create sheet
    sheet_name_hpsn = "Nb HPSN"
    if sheet_name_hpsn in wb.sheetnames:
        del wb[sheet_name_hpsn]
    ws_hpsn = wb.create_sheet(title=sheet_name_hpsn)

    # Write header
    ws_hpsn.append(["Mois"] + list(pivot_hpsn.columns))

    # Write matrix
    for month, row in pivot_hpsn.iterrows():
        ws_hpsn.append([month] + row.tolist())





wb.save(path_CR_histo_regions)

Modelize CR forward

In [3]:
ath_CR_histo_regions = r"M:\PRICING\Pricer - Agrégation\benedrien\Dev Python\Parameters.xlsx"
sheet_source = "Histo CR régions"

# --- Load source data ---
df = pd.read_excel(path_CR_histo_regions, sheet_name=sheet_source)

required_cols = ["libelle_region", "Mois-annee", "CR eol sans hpsn", "CR sol sans hpsn"]
for col in required_cols:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' missing from sheet '{sheet_source}'.")

df[["Year", "Month"]] = df["Mois-annee"].astype(str).str.split("-", expand=True)
df["Year"] = df["Year"].astype(int)
df["Month"] = df["Month"].astype(int)

month_names = {
    1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
    7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"
}
df["Month_name"] = df["Month"].map(month_names)

wb = load_workbook(path_CR_histo_regions)
fill_forecast = PatternFill(start_color="FFFACD", end_color="FFFACD", fill_type="solid")  # yellow for forecast

# --- Function to create a pivot sheet ---
def create_pivot_sheet(pivot_df, sheet_name, last_year=None, slopes=None, highlight_future=False, as_percent=False):
    if sheet_name in wb.sheetnames:
        del wb[sheet_name]
    ws = wb.create_sheet(title=sheet_name)
    ws.append(["Mois"] + list(pivot_df.columns) + ["", "Slope"])
    for i, (month, row) in enumerate(pivot_df.iterrows(), start=2):
        ws.cell(row=i, column=1, value=month)
        for j, (year, val) in enumerate(row.items(), start=2):
            if pd.isna(val):
                continue
            if as_percent:
                val_display = val * 100
            else:
                val_display = val
            cell = ws.cell(row=i, column=j, value=val_display)
            if highlight_future and last_year is not None and year >= last_year:
                cell.fill = fill_forecast
        if slopes is not None:
            slope_val = slopes.get(month)
            if slope_val is not None and as_percent:
                slope_val *= 100
            ws.cell(row=i, column=len(pivot_df.columns) + 3, value=slope_val)

# --- Create CR matrices per region/tech with regression ---
for region in df["libelle_region"].dropna().unique():
    sub = df[df["libelle_region"] == region].copy()

    for tech, col in [("Eolien", "CR eol sans hpsn"), ("Solaire", "CR sol sans hpsn")]:
        pivot = pd.pivot_table(
            sub, index="Month_name", columns="Year", values=col, aggfunc="mean"
        ).reindex(list(month_names.values()))

        if pivot.empty:
            continue

        years = sorted(pivot.columns.tolist())
        last_year = max(years)
        future_years = list(range(last_year + 1, last_year + 6))

        # Add future columns
        for y in future_years:
            pivot[y] = np.nan

        slopes = {}
        for month in pivot.index:
            vals = pivot.loc[month, years].dropna()
            if len(vals) >= 2:
                X = np.array(vals.index).reshape(-1, 1)
                y_vals = vals.values
                model = LinearRegression().fit(X, y_vals)
                slope = model.coef_[0]
                slopes[month] = slope

                # Predict missing in last year + future
                pred_years = [y for y in years + future_years if pd.isna(pivot.loc[month, y])]
                if pred_years:
                    preds = model.predict(np.array(pred_years).reshape(-1, 1))

                    # Cap ±5% relative to last known
                    last_known_year = vals.index.max()
                    last_known_val = vals.loc[last_known_year]
                    preds_capped = []
                    for yhat in preds:
                        delta = (yhat - last_known_val) / last_known_val
                        if delta > 0.05:
                            yhat = last_known_val * 1.05
                        elif delta < -0.05:
                            yhat = last_known_val * 0.95
                        preds_capped.append(yhat)
                    for y_pred, y_val in zip(pred_years, preds_capped):
                        pivot.loc[month, y_pred] = y_val
            else:
                slopes[month] = None

        create_pivot_sheet(pivot, f"{tech}_{region}".replace("/", "_")[:31],
                           last_year=last_year, slopes=slopes,
                           highlight_future=True, as_percent=True)

wb.save(path_CR_histo_regions)

c:\Users\A220925\AppData\Local\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


⚠️ Sheet already exists: Eolien_Auvergne-Rhône-Alpes
⚠️ Sheet already exists: Solaire_Auvergne-Rhône-Alpes
⚠️ Sheet already exists: Eolien_Bourgogne-Franche-Comté
⚠️ Sheet already exists: Solaire_Bourgogne-Franche-Comté
⚠️ Sheet already exists: Eolien_Bretagne
⚠️ Sheet already exists: Solaire_Bretagne
⚠️ Sheet already exists: Eolien_Centre-Val de Loire
⚠️ Sheet already exists: Solaire_Centre-Val de Loire
⚠️ Sheet already exists: Eolien_Grand Est
⚠️ Sheet already exists: Solaire_Grand Est
⚠️ Sheet already exists: Eolien_Hauts-de-France
⚠️ Sheet already exists: Solaire_Hauts-de-France
⚠️ Sheet already exists: Eolien_Normandie
⚠️ Sheet already exists: Solaire_Normandie
⚠️ Sheet already exists: Eolien_Nouvelle-Aquitaine
⚠️ Sheet already exists: Solaire_Nouvelle-Aquitaine
⚠️ Sheet already exists: Eolien_Occitanie
⚠️ Sheet already exists: Solaire_Occitanie
⚠️ Sheet already exists: Eolien_Pays de la Loire
⚠️ Sheet already exists: Solaire_Pays de la Loire
⚠️ Sheet already exists: Eolien_Proven